# EDGI Analysis of the Climate and Economic Justice Screening Tool
## This notebook prepares the primary data for analysis here.

# First, we import some extra sources of Python code to help with the analysis

In [2]:
import pandas
from pandas.errors import EmptyDataError
import urllib
print("Done!")

Done!


# Import custom-built code modules

In [3]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null; # Import our custom modules for handling EPA ECHO data
from ECHO_modules.get_data import get_echo_data # Function for getting our copy of ECHO data mirrored at the Stony Brook University
from ECHO_modules.data_set_presets import ATTRIBUTE_TABLES # A lookup that contains key features (id fields, date fields, etc.) describing ECHO tables
print("Done!")

Done!


# Load previously generated data (optional)

In [57]:
npdes = pandas.read_csv("/content/cwa_final.csv")
#cwa_violations = pandas.read_csv("/content/cwa_violations.csv")
#cwa_ins = pandas.read_csv("/content/cwa_ins.csv")

caa = pandas.read_csv("/content/caa_final.csv")
#caa_violations = pandas.read_csv("/content/caa_violations.csv")
#caa_ins = pandas.read_csv("/content/caa_ins.csv")

rcra = pandas.read_csv("/content/rcra_final.csv")
#rcra_violations = pandas.read_csv("/content/rcra_violations.csv")
#rcra_ins = pandas.read_csv("/content/rcra_ins.csv")

sdwa = pandas.read_csv("/content/sdwa_final.csv")
#sdwa_violations = pandas.read_csv("/content/sdwa_violations.csv")
#sdwa_ins = pandas.read_csv("/content/sdwa_ins.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Prepare our request to the Stony Brook University server to get our copy of EPA ECHO data

In [8]:
# We are going to pull from the ECHO_EXPORTER table, which summarizes recent enforcement and compliance trends for 2+ million regulated facillities
# Here are the columns we want to get from that data table
column_mapping = {
    '"NPDES_FLAG"': str, # Regulated under Clean Water Act National Pollutant Discharge Elimination System
    '"AIR_FLAG"': str, # Regulated under Clean Air Act
    '"RCRA_FLAG"': str, # Regulated under Resource Conservation and Recovery Act
    '"SDWIS_FLAG"': str, # Regulated under Safe Drinking Water Act
    '"REGISTRY_ID"': str, # Official EPA IDs
    '"FAC_DERIVED_CB2010"': str, # The 2010 Census Block the facility is in. EPA hasn't updated for 2020 Census yet. CEJST also uses 2010 Census boundaries.
    '"FAC_MAJOR_FLAG"': str, # A Y/N flag indicating whether a facility has a "major" permit. Basically an indication of the significance of the facility - is it a local Walgreen's or a huge coal-fired power plant.
    '"FAC_NAICS_CODES"': str # The North American Industrial Classification System codes that EPA uses to describe each facility. This will help us determine what kind of facility we're looking at, and to analyze patterns in community "diadvantageness" in terms of sector.
}
column_names = list( column_mapping.keys() )
columns_string = ','.join( column_names )
print("Done!")

Done!


In [67]:
# In this cell, we get EPA's Enforcement and Compliance History Online data from our weekly-updated copy of it at the Stony Brook University.
# Running this cell may take some time since we're going to be asking for 25,000 or so records (rows)!
sql = 'select '+columns_string+' from "ECHO_EXPORTER" where "FAC_MAJOR_FLAG" = \'Y\''
url = 'http://portal.gss.stonybrook.edu/echoepa/?query='
data_location = url+urllib.parse.quote_plus(sql) + '&pg'
echo_data = pandas.read_csv(data_location, dtype=str) # We default to encoding all of the data, even numeric data, as strings. We'll convert later when necessary.
echo_data # We'll show a selection of the data - the first 5 and last 5 rows (facilities)

,NPDES_FLAG,AIR_FLAG,RCRA_FLAG,SDWIS_FLAG,REGISTRY_ID,FAC_DERIVED_CB2010,FAC_MAJOR_FLAG,FAC_NAICS_CODES
0,Y,Y,Y,Y,110007334072,240010022002081,Y,322121 322110 327410
1,Y,Y,N,N,110006792863,10059507002010,Y,321113 321999
2,Y,Y,Y,N,110059765696,311770501023071,Y,325211 311221 325199 325414 325193 325110
3,Y,Y,Y,N,110043798487,470010213024035,Y,221112 326199
4,Y,Y,Y,N,110020689450,390930601001031,Y,562212 211130 221118 221119
...,...,...,...,...,...,...,...,...
25134,N,Y,N,N,110007382037,350150007004184,Y,211130 211112
25135,N,Y,Y,N,110002119216,260810117021030,Y,327420 32742
25136,Y,N,N,N,110040011563,220979611002026,Y,NaN
25137,Y,N,Y,N,110006495337,121030249062019,Y,924110


# EPA has derived the 2010 Census Block that each facility resides within

In [68]:
# First, we need to add 0s back to any blocks that lost it in translation (Alabama, California, etc. have federal id codes - FIPS - that start with 0)
echo_data.loc[echo_data["FAC_DERIVED_CB2010"].str.len() == 14, "FAC_DERIVED_CB2010"] = "0" + echo_data["FAC_DERIVED_CB2010"]
# Then we shorten each CB code to its 10 digit Census tract code.
echo_data["FAC_DERIVED_CB2010"] = echo_data["FAC_DERIVED_CB2010"].str.slice(0,11)
echo_data

,NPDES_FLAG,AIR_FLAG,RCRA_FLAG,SDWIS_FLAG,REGISTRY_ID,FAC_DERIVED_CB2010,FAC_MAJOR_FLAG,FAC_NAICS_CODES
0,Y,Y,Y,Y,110007334072,24001002200,Y,322121 322110 327410
1,Y,Y,N,N,110006792863,01005950700,Y,321113 321999
2,Y,Y,Y,N,110059765696,31177050102,Y,325211 311221 325199 325414 325193 325110
3,Y,Y,Y,N,110043798487,47001021302,Y,221112 326199
4,Y,Y,Y,N,110020689450,39093060100,Y,562212 211130 221118 221119
...,...,...,...,...,...,...,...,...
25134,N,Y,N,N,110007382037,35015000700,Y,211130 211112
25135,N,Y,Y,N,110002119216,26081011702,Y,327420 32742
25136,Y,N,N,N,110040011563,22097961100,Y,NaN
25137,Y,N,Y,N,110006495337,12103024906,Y,924110


# Set up a function for getting the program-specific ID codes for each of these facilities

In [17]:
# Each facility regulated under each program (CWA, CAA, RCRA) has at least one code that is specific to the program
def get_program_ids(program):
  pgm_data = echo_data.loc[echo_data[program+"_FLAG"] == 'Y'] #AIR, NPDES, RCRA, SDWIS
  
  if program == "SDWIS":
    program = "SDWA" # Lookup in ECHO_EXPORTER is SDWIS but in EXP_PGM it's SDWA

  reg_ids = list(pgm_data["REGISTRY_ID"].unique()) # Registry IDs to look up to get program IDs

  batchsize = 50 # batch the request to the SBU server.

  pgm_ids = pandas.DataFrame() # End result

  # Could take quite a while!
  for i in range(0, len(reg_ids), batchsize):
      batch = reg_ids[i:i+batchsize]

      id_string = ""
      for id in batch:
        id_string += "'"+str(id)+"',"
      id_string = id_string[:len(id_string)-1]
      
      try:
        sql = 'select * from "EXP_PGM" where "PGM" like \'{}_IDS\' and "REGISTRY_ID" in ({})'.format(program, id_string)
        url = 'http://portal.gss.stonybrook.edu/echoepa/?query='
        data_location = url+urllib.parse.quote_plus(sql) + '&pg'
        #print(sql)
        df = pandas.read_csv(data_location)
        pgm_ids = pgm_ids.append(df)
      except EmptyDataError:
        print("...")

  return pgm_ids
print("Done!")

Done!


# Set up a function for getting program-specific data for each facility

In [12]:
# This function will use program ids to get specific violations and inspections information for each facility
def get_program_data(table, ids):

  table_name = ATTRIBUTE_TABLES[table]["table_name"]
  idx_field = ATTRIBUTE_TABLES[table]['idx_field']

  batchsize = 50 # batch the request to the SBU server.

  pgm_results = pandas.DataFrame() # End result

  # Could take quite a while!
  for i in range(0, len(ids), batchsize):
      batch = ids[i:i+batchsize]
      
      id_string = ""
      for id in batch:
        id_string += "'"+id+"',"
      id_string = id_string[:len(id_string)-1]

      try:
        sql = 'select * from "{}" where "{}" in ({})'.format(table_name, idx_field, id_string)
        url = 'http://portal.gss.stonybrook.edu/echoepa/?query='
        data_location = url+urllib.parse.quote_plus(sql) + '&pg'
        #print(sql)
        df = pandas.read_csv(data_location)
        pgm_results = pgm_results.append(df)
      except EmptyDataError:
        print("...")

  return pgm_results
print("Done!")

Done!


# Set up a program for summarizing the program-specific information

In [13]:
# This function will summarizing the program-specific information
def pgm_summarize(df, table):
  if table == "CWA Violations":
    year = df["YEARQTR"].astype("str").str[0:4:1] # In this table, years are formatted by quarter e.g. 20141 for the first quarter of 2014 so we just eliminate the quarter indicator
    df["YEARQTR"] = year
  if table == "CAA Violations":
    df['Date'] = df['EARLIEST_FRV_DETERM_DATE'].fillna(df['HPV_DAYZERO_DATE']) # Some CAA violations records don't have one or the other date information. We use the best available.
  
  df[ATTRIBUTE_TABLES[table]["date_field"]] = pandas.to_datetime(df[ATTRIBUTE_TABLES[table]["date_field"]], 
    format=ATTRIBUTE_TABLES[table]["date_format"], errors='coerce')    
  
  df_x = df.loc[df[ATTRIBUTE_TABLES[table]["date_field"]].dt.strftime('%Y') > '2000'] # EPA says only data from 2001 and later is reliable
  df_x = df_x.groupby(by=[ATTRIBUTE_TABLES[table]["idx_field"]]).agg({ATTRIBUTE_TABLES[table]["agg_col"]: ATTRIBUTE_TABLES[table]["agg_type"]})
  
  return df_x
print("Done!")

Done!


# Get information - NPDES

In [58]:
npdes = get_program_ids("NPDES") # First, get IDs
npdes.to_csv("cwa_ids.csv") # Save data

# Violations
cwa_violations = get_program_data("CWA Violations",list(npdes["PGM_ID"].unique()))
cwa_violations.to_csv("cwa_violations.csv") # Save data
cwa_violations_grouped = pgm_summarize(cwa_violations, "CWA Violations") # Violations will be summarized under "NUME90Q" Note that we are only counting effluent violations - not paperwork violations.

# Inspections
cwa_ins = get_program_data("CWA Inspections",list(npdes["PGM_ID"].unique()))
cwa_ins.to_csv("cwa_ins.csv") # Save data
cwa_ins_grouped = pgm_summarize(cwa_ins, "CWA Inspections") # Inspections information will be summarized under "STATE_EPA_FLAG"

# Merge with NPDES lookup
npdes.set_index("PGM_ID", inplace=True)
npdes = npdes.join(cwa_violations_grouped)
npdes = npdes.join(cwa_ins_grouped)
npdes.to_csv("cwa_final.csv") # NPDES results
npdes

,Unnamed: 0,PGM,REGISTRY_ID,NUME90Q,STATE_EPA_FLAG
PGM_ID,,,,,
WA0032221,0,NPDES_IDS,110000897085,7.0,1.0
VAR050936,1,NPDES_IDS,110020668507,NaN,NaN
VA0006408,2,NPDES_IDS,110020680306,9.0,19.0
VAN040073,3,NPDES_IDS,110020680306,0.0,1.0
ARR000405,4,NPDES_IDS,110031104622,0.0,NaN
...,...,...,...,...,...
MSL042455,27,NPDES_IDS,110000569545,0.0,NaN
PA0021148,28,NPDES_IDS,110001100264,37.0,28.0
MI0021555,29,NPDES_IDS,110006740839,92.0,21.0


# Get information - CAA

In [59]:
caa = get_program_ids("AIR") # First, get IDs
caa.to_csv("caa_ids.csv") # Save data

# Violations
caa_violations = get_program_data("CAA Violations",list(caa["PGM_ID"].unique()))
caa_violations.to_csv("caa_violations.csv") # Save data
caa_violations_grouped = pgm_summarize(caa_violations, "CAA Violations") # Violations information will be summarized under "AGENCY_TYPE_DESC"

#Inspections
caa_ins = get_program_data("CAA Inspections",list(caa["PGM_ID"].unique()))
caa_ins.to_csv("caa_ins.csv") # Save data
caa_ins_grouped = pgm_summarize(caa_ins, "CAA Inspections") # Inspections information will be summarized under "STATE_EPA_FLAG"

# Merge with CAA lookup
caa.set_index("PGM_ID", inplace=True) 
caa = caa.join(caa_violations_grouped) 
caa = caa.join(caa_ins_grouped) 
caa.to_csv("caa_final.csv") # CAA results
caa

,Unnamed: 0,PGM,REGISTRY_ID,AGENCY_TYPE_DESC,STATE_EPA_FLAG
PGM_ID,,,,,
SC00005600008,0,AIR_IDS,110016966381,13.0,100.0
SC00005600164,1,AIR_IDS,110016966381,1.0,27.0
SC00005600244,2,AIR_IDS,110016966381,NaN,22.0
IALIN0001911300050,3,AIR_IDS,110017344337,NaN,64.0
OH0000000247000760,4,AIR_IDS,110020689450,1.0,28.0
...,...,...,...,...,...
TX0000004818100056,34,AIR_IDS,110007206647,NaN,7.0
NC0000003701900135,35,AIR_IDS,110070815063,1.0,1.0
TX0000004846500001,36,AIR_IDS,110033181296,NaN,4.0


# Get information - RCRA

In [60]:
rcra = get_program_ids("RCRA") # First, get IDs
rcra.to_csv("rcra_ids.csv") # Save data

# Violations
rcra_violations = get_program_data("RCRA Violations",list(rcra["PGM_ID"].unique()))
rcra_violations.to_csv("rcra_violations.csv") # Save data
rcra_violations_grouped = pgm_summarize(rcra_violations, "RCRA Violations") # Violations information will be summarized under "VIOL_DETERMINED_BY_AGENCY"

# Inspections
rcra_ins = get_program_data("RCRA Inspections",list(rcra["PGM_ID"].unique()))
rcra_ins.to_csv("rcra_ins.csv") # Save data
rcra_ins_grouped = pgm_summarize(rcra_ins, "RCRA Inspections") # Inspections information will be summarized under "EVALUATION_AGENCY"

# Merge with RCRA lookup
rcra.set_index("PGM_ID", inplace=True)
rcra = rcra.join(rcra_violations_grouped) 
rcra = rcra.join(rcra_ins_grouped) 
rcra.to_csv("rcra_final.csv") # RCRA results
rcra

,Unnamed: 0,PGM,REGISTRY_ID,VIOL_DETERMINED_BY_AGENCY,EVALUATION_AGENCY
PGM_ID,,,,,
LAD057117434,0,RCRA_IDS,110000613747,50.0,33.0
TXD074195678,1,RCRA_IDS,110000619590,63.0,23.0
NYD055735807,2,RCRA_IDS,110000884927,NaN,2.0
WAD009249616,3,RCRA_IDS,110000897085,NaN,NaN
NYD980645774,4,RCRA_IDS,110001566421,5.0,3.0
...,...,...,...,...,...
NCD042892067,7,RCRA_IDS,110000346803,10.0,88.0
OHR000034025,8,RCRA_IDS,110004731880,22.0,49.0
MIK147134953,9,RCRA_IDS,110006740839,NaN,NaN


# Get information - SDWA

In [62]:
sdwa = get_program_ids("SDWIS") # First, get IDs
sdwa.to_csv("sdwa_ids.csv") # Save data

# Violations
sdwa_violations = get_program_data("SDWA Serious Violators",list(sdwa["PGM_ID"].unique()))
sdwa_violations.to_csv("sdwa_violations.csv") # Save data
sdwa_violations_grouped = pgm_summarize(sdwa_violations, "SDWA Serious Violators") # Violations information will be summarized under "PWS_SIZE"

# Inspections
sdwa_ins = get_program_data("SDWA Site Visits",list(sdwa["PGM_ID"].unique()))
sdwa_ins.to_csv("sdwa_ins.csv") # Save data
sdwa_ins_grouped = pgm_summarize(sdwa_ins, "SDWA Site Visits") # Inspections information will be summarized under "PWS_SIZE"

# Merge with SDWA lookup
sdwa.set_index("PGM_ID", inplace=True)
sdwa = sdwa.join(sdwa_violations_grouped, rsuffix="_Violations_Count") 
sdwa = sdwa.join(sdwa_ins_grouped, rsuffix="_Inspections_Count") 
sdwa.to_csv("sdwa_final.csv") # SDWA results
sdwa

,Unnamed: 0,PGM,REGISTRY_ID,PWS_SIZE,PWS_SIZE_Inspections_Count
PGM_ID,,,,,
TX1780017,0,SDWA_IDS,110000465078,NaN,4.0
IN2870801,1,SDWA_IDS,110000602045,NaN,8.0
NC0425035,2,SDWA_IDS,110000604301,NaN,18.0
MD1160010,3,SDWA_IDS,110005974286,NaN,4.0
NM3518025,4,SDWA_IDS,110011026941,1.0,5.0
...,...,...,...,...,...
GA0950023,54,SDWA_IDS,110000361000,NaN,NaN
TX0290002,55,SDWA_IDS,110039693702,5.0,5.0
AK2333039,56,SDWA_IDS,110041963266,NaN,2.0


# Merge these results with the full ECHO data

In [69]:
# Each row in the output will be a facility, with a summary of its CAA/CWA/RCRA violations and inspections over the past 20 years
echo_data.set_index("REGISTRY_ID", inplace=True)

# NPDES
npdes = npdes.groupby(by=["REGISTRY_ID"]).sum() # sum CWA violations, inspections by Registry_ID (important for accounting for multiple PGM_IDS)
npdes.index = npdes.index.astype(str)
echo_data = echo_data.join(npdes, rsuffix="_CWA")

# CAA
caa = caa.groupby(by=["REGISTRY_ID"]).sum() # sum CAA violations, inspections by Registry ID
caa.index = caa.index.astype(str)
echo_data = echo_data.join(caa, rsuffix="_CAA")

# RCRA
rcra = rcra.groupby(by=["REGISTRY_ID"]).sum() # sum RCRA violations, inspections by Registry ID
rcra.index = rcra.index.astype(str)
echo_data = echo_data.join(rcra, rsuffix="_RCRA")

# SDWA
sdwa = sdwa.groupby(by=["REGISTRY_ID"]).sum() # sum SDWA violations, inspections by Registry ID
sdwa.index = sdwa.index.astype(str)
echo_data = echo_data.join(sdwa, rsuffix="_SDWA")

echo_data['FAC_NAICS_CODES'] = echo_data['FAC_NAICS_CODES'].astype(str) # Process this column for later aggregation

echo_data

,NPDES_FLAG,AIR_FLAG,RCRA_FLAG,SDWIS_FLAG,FAC_DERIVED_CB2010,FAC_MAJOR_FLAG,FAC_NAICS_CODES,Unnamed: 0,NUME90Q,STATE_EPA_FLAG,Unnamed: 0_CAA,AGENCY_TYPE_DESC,STATE_EPA_FLAG_CAA,Unnamed: 0_RCRA,VIOL_DETERMINED_BY_AGENCY,EVALUATION_AGENCY,Unnamed: 0_SDWA,PWS_SIZE,PWS_SIZE_Inspections_Count
REGISTRY_ID,,,,,,,,,,,,,,,,,,,
110000307766,N,Y,N,N,72097081512,Y,49312 311111 311711,NaN,NaN,NaN,36.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
110000307775,Y,Y,Y,Y,72113072300,Y,312140 221310,30.0,0.0,1.0,11.0,0.0,23.0,12.0,4.0,2.0,33.0,0.0,0.0
110000307800,Y,Y,N,Y,72123953200,Y,221112 221122 221119 221121,60.0,436.0,16.0,10.0,0.0,86.0,NaN,NaN,NaN,87.0,7.0,5.0
110000307837,Y,N,Y,N,72137121802,Y,321114,13.0,0.0,1.0,NaN,NaN,NaN,27.0,6.0,7.0,NaN,NaN,NaN
110000307855,N,Y,Y,N,72153750300,Y,333999 339999 325221 325220,NaN,NaN,NaN,27.0,1.0,18.0,73.0,3.0,17.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110071156097,Y,N,N,N,04019002501,Y,nan,56.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110071176777,N,Y,N,N,37079950101,Y,562213,NaN,NaN,NaN,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
NaN,N,Y,N,N,NaN,Y,211120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Summarize this facility-specific information at the Census tract level

In [76]:
final_echo_data = echo_data.groupby("FAC_DERIVED_CB2010").agg({
    "NUME90Q": "sum", # CWA violations
    "AGENCY_TYPE_DESC": "sum", # CAA violations
    "VIOL_DETERMINED_BY_AGENCY": "sum", # RCRA violations
    "PWS_SIZE": "sum", # SDWA violations
    "STATE_EPA_FLAG": "sum", # CWA inspections
    "STATE_EPA_FLAG_CAA": "sum", # CAA inspections _CAA
    "EVALUATION_AGENCY": "sum", # RCRA inspections
    "PWS_SIZE_Inspections_Count": "sum", # SDWA inspections
    "FAC_MAJOR_FLAG": "count", # Count of regulalted major facilities in the tract
    "FAC_NAICS_CODES": ' '.join # Join together the NAICS codes
}) # sum of violations, count of facilities

# Rename columns for clarity
final_echo_data = final_echo_data.rename(columns={
    "NUME90Q": "CWA Violations", # CWA violations
    "AGENCY_TYPE_DESC": "CAA Violations", # CAA violations
    "VIOL_DETERMINED_BY_AGENCY": "RCRA Violations", # RCRA violations
    "PWS_SIZE": "SDWA Violations", # SDWA violations
    "STATE_EPA_FLAG": "CWA Inspections", # CWA inspections
    "STATE_EPA_FLAG_CAA": "CAA Inspections", # CAA inspections _CAA
    "EVALUATION_AGENCY": "RCRA Inspections", # RCRA inspections
    "PWS_SIZE_Inspections_Count": "SDWA Inspections", # SDWA inspections
    "FAC_MAJOR_FLAG": "Facilities", # Count of regulalted major facilities in the tract
    "FAC_NAICS_CODES": "FAC_NAICS_CODES"
})

# Violations
final_echo_data["Total Violations Since 2001"] = final_echo_data["CWA Violations"] + final_echo_data["CAA Violations"] + final_echo_data["RCRA Violations"]  + final_echo_data["SDWA Violations"]
final_echo_data["Violations Per Facility Since 2001"] = final_echo_data["Total Violations Since 2001"] / final_echo_data["Facilities"]

# Inspections
final_echo_data["Total Inspections Since 2001"] = final_echo_data["CWA Inspections"] + final_echo_data["CAA Inspections"] + final_echo_data["RCRA Inspections"] + final_echo_data["SDWA Inspections"]
final_echo_data["Inspections Per Facility Since 2001"] = final_echo_data["Total Inspections Since 2001"] / final_echo_data["Facilities"]

# Save the data
final_echo_data.to_csv("echo_data.csv")

final_echo_data

,CWA Violations,CAA Violations,RCRA Violations,SDWA Violations,CWA Inspections,CAA Inspections,RCRA Inspections,SDWA Inspections,Facilities,FAC_NAICS_CODES,Total Violations Since 2001,Violations Per Facility Since 2001,Total Inspections Since 2001,Inspections Per Facility Since 2001
FAC_DERIVED_CB2010,,,,,,,,,,,,,,
01001020700,21.0,0.0,5.0,0.0,29.0,31.0,5.0,0.0,2,326112 323111 326111 221320,26.0,13.0,65.0,32.5
01001020801,1.0,0.0,0.0,0.0,32.0,89.0,1.0,11.0,2,221112 22111 322130 325194 32519 322121 325191...,1.0,0.5,133.0,66.5
01001021000,2.0,0.0,0.0,0.0,8.0,82.0,0.0,0.0,1,221112 237990,2.0,2.0,90.0,90.0
01003010300,11.0,0.0,16.0,0.0,11.0,0.0,37.0,0.0,1,325199,27.0,27.0,48.0,48.0
01003010500,51.0,0.0,12.0,0.0,37.0,18.0,39.0,0.0,2,nan 321114,63.0,31.5,94.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151950600,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,2,311311 324191,0.0,0.0,8.0,4.0
72151951000,569.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1,22132,569.0,569.0,36.0,36.0
72151951300,183.0,0.0,0.0,1.0,17.0,84.0,0.0,0.0,1,324110 81112 32411 324191 424710 493110 492210,184.0,184.0,101.0,101.0
